## Our task is to classify spam/ham messages.



## 0 - Download data

In [1]:
!pip install wget
import wget
wget.download('https://dru.fra1.digitaloceanspaces.com/DS_Fundamentals/datasets/04_supervised_learning/Naive_Bayes_Classifier/spam.csv')

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=98acc95db74de281f3e7f1dd0536bac5ab95a72d03c0b0c4f85768156c0b1c2b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


'spam.csv'

## 1 - Packages ##

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter

## 2 - Overview of the Problem set ##

In [3]:
def load_data():
    df = pd.read_csv('spam.csv', encoding='latin-1')
    df_for_tests = df.head()

    idx = np.arange(df.shape[0])
    np.random.shuffle(idx)

    train_set_size = int(df.shape[0] * 0.8)

    train_set = df.loc[idx[:train_set_size]]
    test_set = df.loc[idx[train_set_size:]]

    return train_set, test_set, df_for_tests

In [4]:
train_set, test_set, df_for_tests = load_data()
print(train_set)

        v1                                                 v2
3600   ham                       Jay told me already, will do
3280   ham  I tot it's my group mate... Lucky i havent rep...
1936   ham      My planning usually stops at \find hella weed
157    ham  Hello, my love. What are you doing? Did you ge...
3439   ham                     What time you thinkin of goin?
...    ...                                                ...
3314  spam  FREE MESSAGE Activate your 500 FREE Text Messa...
1493   ham  How are you with moneY...as in to you...money ...
1516   ham  I need to come home and give you some good lov...
3664   ham              Ha... U jus ate honey ar? So sweet...
2911   ham  You didn't have to tell me that...now i'm thin...

[4457 rows x 2 columns]


## 3 - Naive Bayes Classifier
**Mathematical expression of the algorithm**:


This algorithm is based on Bayes' theorem:
    \begin{equation}
    P(A_{j}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{1},\dots,x_{n}) = \frac{P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})P(A_{j})}{P(x_{1},\dots,x_{n})}
    \end{equation}
    
Ignoring denominator (because it stays the same for all cases):

$$ \begin{equation}
    P(A_{j})P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j}) = P(A_{j}, x_{1},\dots,x_{n}) = \\
  \hspace{1cm} = P(x_{1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{2},\dots,x_{n}, A_{j})P(x_{2}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{3}, \dots ,x_{n}, A_{j})\dots P(x_{n-1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{n}, A_{j}) \approx \\
  \hspace{1cm}
  \end{equation}$$
By making an assumption that the $x_{i}$ are conditionally independent of each other:
$$ \begin{equation} \approx P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})
   \end{equation}$$
   
We can calculate the probability, if we know the prior probability:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big(P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})\big)
   \end{equation}$$
   
   
Due to floating point underflow, the above is usually replaced with the numerically tractable expression:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big( \ln(P(A_{j})) + \sum_{i=1}^{n} \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) \big)
   \end{equation}$$
   
For more consistent knowledge of the NBC algorithm (https://web.stanford.edu/~jurafsky/slp3/4.pdf)

**Training the Naive Bayes Classifier**:

How can we find the probabilities $\ln(P(A_{j}))$ and $\ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j}))$ ? We'll simply use the frequencies in the data. For the class prior $P(A_{j})$ we find what percentage of the messages in our training set are in each class $A_{j}$:

$$ \begin{equation}
    \ln(P(A_{j})) = \ln\big(\frac{N_{A_{j}}}{N}\big)
    \tag{1}
   \end{equation}$$

where $N_{A_{j}}$ is the number of messages in our training data with class $A_{j}$ and $N$ be the total number of messages.

In $P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})$ we just compute as the fraction of times the word $x_{i}$ appears among all words in all messages of class $A_{j}$:

$$ \begin{equation}
    \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) = \ln\big(\frac{ N_{x_i, A_j}}{|A_{j}|} \big)
    \tag{2}
   \end{equation}
$$

where $N_{x_i, A_j}$ is number of times word $x_{i}$ appears in messages from class $A_{j}$ and $|A_{j}|$ - total count of all words in class $A_{j}$.

   
#### Laplace smoothing

In statistics, additive smoothing, also called Laplace smoothing, or Lidstone smoothing, is a technique that is used to smooth categorical data. Given an observation
$\begin{equation}
    message = (x_{1}\, \dots \,x_{n})
 \end{equation}$, a "smoothed" version of the data gives the estimator:


$$ \begin{equation}
    \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) = \ln\big(\frac{ N_{x_i, A_j} + \alpha}{ |A_{j}| +  \alpha * |V|} \big)
    \tag{3}
   \end{equation}
$$

where the pseudocount
$\begin{equation}
    \alpha > 0
 \end{equation}$ is the smoothing parameter (
$\begin{equation}
    \alpha = 0
 \end{equation}$ corresponds to no smoothing) and $V$ is a vocabulary, which consists of the union of all the words in all classes.

### 3.1 - Preprocessing the data

Let's clean our data and leave only letters a-z, A-Z, numbers 0-9 and cast all letters to lowercase, replace double to n spaces with just one space, remove trailing spaces.

In [5]:
# Clean the data

def clean_data(message):

    message = message.lower()
    cleaned_message = ''.join(char if char.isalnum() or char.isspace() else ' ' for char in message)
    cleaned_message = ' '.join(cleaned_message.split())
    return cleaned_message

In [6]:
sentence_1 = 'Doesn\'t get, how{to}% \\operate+66.7 :after[it]"" & lt;# & gt; won\'t `or(what)'
sentence_2 = 'O\]k,.lar7i$double{} check wif*& da! hair: [dresser;   ..already He SaID-77.88.5 wun cut v short question(std txt rate)T&C\'s'
print('cleaned: ',clean_data(sentence_1))
print('cleaned: ',clean_data(sentence_2))

cleaned:  doesn t get how to operate 66 7 after it lt gt won t or what
cleaned:  o k lar7i double check wif da hair dresser already he said 77 88 5 wun cut v short question std txt rate t c s


In [7]:
# Preparation data for model

def prep_for_model(train_set, test_set):


    train_set = train_set.copy()
    test_set = test_set.copy()

    train_set['cleaned_message'] = (train_set['v2']).apply(clean_data)
    test_set['cleaned_message'] = (test_set['v2']).apply(clean_data)

#     print(test_set)

    train_set_x = np.array(train_set['cleaned_message'].apply(str.split))
#     train_set_x = [item for sublist in train_set_x for item in sublist]

#     print(train_set_x)

    test_set_x = np.array(test_set['cleaned_message'].apply(str.split))
#     test_set_x = [item for sublist in test_set_x for item in sublist]

    train_set_y = np.array(train_set['v1'])
    test_set_y = np.array(test_set['v1'])

    return train_set_x, train_set_y, test_set_x, test_set_y

train_set_x, train_set_y, test_set_x, test_set_y = prep_for_model(train_set, test_set)

In [8]:
a1, a2, b1, b2 = prep_for_model(df_for_tests.head(3), df_for_tests.tail(2))
print(a2[0], a1[0])
print(b2[0], b1[0])

ham ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
ham ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']


Now let's check words in each category

In [9]:
# Check words in categories

def categories_words(x_train, y_train):

    all_words_list = []
    ham_words_list = []
    spam_words_list = []

    for i in range(len(x_train)):
        word = x_train[i]
        label = y_train[i]

        all_words_list.extend(word)

        if label == 'ham':
            ham_words_list.extend(word)
        elif label == 'spam':
            spam_words_list.extend(word)

    all_words_list = np.array(all_words_list)
    ham_words_list = np.array(ham_words_list)
    spam_words_list = np.array(spam_words_list)


    return all_words_list, ham_words_list, spam_words_list

all_words_list_a1, ham_words_list_a1, spam_words_list_a1 = categories_words(a1, a2)

In [10]:
print('first five "ham" words of a1: ', ham_words_list_a1[:5])

first five "ham" words of a1:  ['go' 'until' 'jurong' 'point' 'crazy']


Let's calculate probability of each word in a category, use

---

(3) formula.

In [11]:
# Calculate log probability of each word in a category using smoothing
def calc_words_probs_of_category(category_words_list, all_words_list, alpha):

    category_words_dict = {}

    word_counts = {}
    num_category_words = 0

    # Count occurrences of words in the specific category
    for word in category_words_list:
        word_counts[word] = word_counts.get(word, 0) + 1
        num_category_words += 1

    num_unique_all_words = len(set(all_words_list))

    # Calculate log probabilities for each word
    for word in all_words_list:
        w_count = word_counts.get(word, 0)
        prob = np.log((w_count + alpha) / (num_category_words + alpha * num_unique_all_words))
        category_words_dict[word] = prob
    # word_counts = Counter(category_words_list)
    # num_unique_category_words = len(set(category_words_list))
    # num_unique_all_words = len(set(all_words_list))

    # for w in all_words_list:
    #     w_count = word_counts[w]
    #     prob = np.log((w_count + alpha)/(num_unique_category_words + alpha*num_unique_all_words))
    #     category_words_dict[w] = prob

    return category_words_dict

In [12]:
ham_words_dict_a1 = calc_words_probs_of_category(ham_words_list_a1, all_words_list_a1, alpha=1)
print('The log prob of the word “87121” from the "ham" category in a1:', ham_words_dict_a1['87121'])

spam_words_dict_a1 = calc_words_probs_of_category(spam_words_list_a1, all_words_list_a1, alpha=1)
print('\nThe log prob of the word “87121” from the "spam" category in a1:', spam_words_dict_a1['87121'])

The log prob of the word “87121” from the "ham" category in a1: -4.3694478524670215

The log prob of the word “87121” from the "spam" category in a1: -3.7612001156935624


Сalculating the prior probability for each class category, use (1) formula.

In [13]:
# Calculate prior log probability of each category
def calc_prior_category_prob(y_train):

    unique_categories, category_counts = np.unique(y_train, return_counts=True)

    ham_index = np.where(unique_categories == 'ham')[0][0]
    spam_index = np.where(unique_categories == 'spam')[0][0]

    count_ham = category_counts[ham_index] if 'ham' in unique_categories else 0
    count_spam = category_counts[spam_index] if 'spam' in unique_categories else 0

    prior_ham_prob = np.log(count_ham / len(y_train)) if len(y_train) > 0 else 0
    prior_spam_prob = np.log(count_spam / len(y_train)) if len(y_train) > 0 else 0
    # category_counts = Counter(y_train)
    # count_ham = category_counts['ham']
    # count_spam = category_counts['spam']
    # prior_ham_prob = np.log(count_ham/len(y_train))
    # prior_spam_prob = np.log(count_spam/len(y_train))
    # return prior_ham_prob, prior_spam_prob

    return prior_ham_prob, prior_spam_prob

In [14]:
print('Prior log probability of "ham" category in a2: ', calc_prior_category_prob(a2)[0])
print('Prior log probability of "spam" category in a2: ', calc_prior_category_prob(a2)[1])

Prior log probability of "ham" category in a2:  -0.40546510810816444
Prior log probability of "spam" category in a2:  -1.0986122886681098


### 3.2 Model

In [15]:
class Naive_Bayes(object):
    """
    Parameters:
    -----------
    alpha: int
        The smoothing coeficient.
    """
    def __init__(self, alpha):
        self.alpha = alpha

        self.train_set_x = None
        self.train_set_y = None

        self.all_words_list = []
        self.ham_words_list = []
        self.spam_words_list = []

        self.ham_words_dict = {}
        self.spam_words_dict = {}

        self.prior_ham_prob = None
        self.prior_spam_prob = None


    def fit(self, train_set_x, train_set_y):

        # Generate all_words_list, ham_words_list, spam_words_list using function 'categories_words';
        # Calculate probability of each word in both categories using function 'calc_words_probs_of_category';
        # Calculate prior probability of each category using function 'calc_prior_category_prob'.
        all_words_list, ham_words_list, spam_words_list = categories_words(train_set_x, train_set_y)
        self.ham_probs = calc_words_probs_of_category(ham_words_list, all_words_list, self.alpha)
        self.spam_probs = calc_words_probs_of_category(spam_words_list, all_words_list, self.alpha)
        self.prior_ham_prob, self.prior_spam_prob = calc_prior_category_prob(train_set_y)


    def predict(self, test_set_x):

        # Calculate probabilities of belonging to ham and spam category
        # Compare these probabilities and choose the max
        # Return list of predicted labels for test set; type(prediction) -> list, len(prediction) = len(test_set_y)
        prediction = []
        for message in test_set_x:
            ham_prob = self.prior_ham_prob + sum(self.ham_probs[word] for word in message if word in self.ham_probs)
            spam_prob = self.prior_spam_prob + sum(self.spam_probs[word] for word in message if word in self.spam_probs)
            label = 'ham' if ham_prob > spam_prob else 'spam'
            prediction.append(label)

        return prediction

## 4 - Training

First of all, we should define a smoothing coeficient (`alpha`).

In [16]:
a = 1

Now we can initialize our model:

In [17]:
model = Naive_Bayes(alpha=a)

Let's train our model:

In [18]:
model.fit(train_set_x, train_set_y)

## 5 - Making predictions

In [19]:
y_predictions = model.predict(test_set_x)

Let's calculate accuracy (accuracy of model must be >0.95):

In [20]:
actual = list(test_set_y)
accuracy = (y_predictions == test_set_y).mean()
print(accuracy)

0.9856502242152466


## 6 - Conclusion
As we can see, our model fits well the hypothesis function to the data.

#### What's next:
1. Try experimenting with the `alpha` to see how this affects the model you have built.
2. Compare the results you have obtained with the `sklearn.naive_bayes.MultinomialNB` model.
3. Try this model in the wild! Select your favorite dataset [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=small&filetype=all&license=all&tagids=13303) and play with it.

##### Naive Bayes Classifier Done!